In [ ]:
# from src.commons.react_agent_factory import create_react_agent
from src.agents.demo_agent.graph import make_demo_agent
from langchain_core.messages import HumanMessage
from src.commons.react_agent_factory import AgentState
from dotenv import load_dotenv
load_dotenv('.env')

True

In [2]:

agent = make_demo_agent()
state = AgentState(messages=[
    HumanMessage(content = 'hello, what can you do?')
])


In [3]:
from src.observability.instrumentation import PerfettoLogger, PerfettoTracer, LangGraphInstrumentationHandler
tracer = PerfettoTracer()
logger = PerfettoLogger()
instrumentation_callback_handler = LangGraphInstrumentationHandler(tracer, logger)

In [4]:
chat_history = []

In [5]:
query_1 = (
    "Check the weather in Paris. "
    "Then, calculate 3 raised to the power of 4. "
    "Finally, reverse the string 'PERFETTO'."
)
chat_history.append(HumanMessage(content=query_1))

# Run Agent
response_1 = await agent.ainvoke(
    {"messages": chat_history}, 
    config={"callbacks": [instrumentation_callback_handler]}
)

# Update history with the result
# The agent returns a dict with 'messages'. We extract the new ones.
new_messages = response_1["messages"]
chat_history = new_messages # In a real app, you'd merge lists or use a Checkpointer

query_2 = "Add the number you calculated earlier to the temperature in Paris."
chat_history.append(HumanMessage(content=query_2))

response_2 = await agent.ainvoke(
    {"messages": chat_history}, 
    config={"callbacks": [instrumentation_callback_handler]}
)
filename = "/tmp/tracer_output.json"
tracer.save(filename)

23:07:14 | 🟢 [CHAIN] LangGraph
23:07:14 |    📝 STATE [call_model]: Input -> {'messages': [{'type': 'human', 'name': None, 'content': "Check the weather in Paris. Then, calculat...
23:07:14 | 🟢 [GRAPH_NODE] Node: call_model
23:07:14 | 🟢 [LLM] LLM: gpt-4o
23:07:15 | 🔴 [LLM] LLM: gpt-4o
23:07:15 |    📝 STATE [call_model]: Input -> {'messages': [{'id': 'cd124ec3-d562-487f-b32a-c5641622a8e7', 'type': 'human'}, {'type': 'ai', 'name'...
23:07:15 | 🟢 [GRAPH_NODE] Node: call_model
23:07:15 | 🔴 [GRAPH_NODE] Node: call_model
23:07:15 | 🔴 [GRAPH_NODE] Node: call_model
23:07:15 |    📝 STATE [tools]: Input -> {'messages': [{'id': 'cd124ec3-d562-487f-b32a-c5641622a8e7', 'type': 'human'}, {'id': 'lc_run--019bd...
23:07:15 | 🟢 [GRAPH_NODE] Node: tools
23:07:15 | 🟢 [TOOL] Tool: get_weather
23:07:15 | 🟢 [TOOL] Tool: reverse_text
23:07:15 | 🟢 [TOOL] Tool: complex_calculation
23:07:15 | 🔴 [TOOL] Tool: reverse_text
23:07:15 | 🔴 [TOOL] Tool: get_weather
23:07:16 | 🔴 [TOOL] Tool: complex_calculation
23:07:16 

In [7]:
response_2

{'messages': [HumanMessage(content="Check the weather in Paris. Then, calculate 3 raised to the power of 4. Finally, reverse the string 'PERFETTO'.", additional_kwargs={}, response_metadata={}, id='cd124ec3-d562-487f-b32a-c5641622a8e7'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 68, 'prompt_tokens': 225, 'total_tokens': 293, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_deacdd5f6f', 'id': 'chatcmpl-CzVDOcrgbDSpmfWk7orgjrQbV6UqO', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019bd325-e372-7623-900a-25a3684e4f69-0', tool_calls=[{'name': 'get_weather', 'args': {'city': 'Paris'}, 'id': 'call_qQTNngoOUQImF9AIU6xtyaV1', 'type': 't